To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 🔍 Smart Filtering

**Hook:** *"Filter data by meaning, not just exact matches"*

Traditional filters need exact conditions - "status = 'urgent'" or "category = 'bug'". But what about "sounds frustrated" or "seems like a feature request"? Smart filtering uses AI to understand context and intent, finding records that match semantic criteria impossible with SQL.

**What you'll see in this 2-minute demo:**
- 📝 **Customer feedback data** - Mixed complaints, requests, and praise
- 🧠 **Semantic filters** - "Find angry customers" or "show feature requests"
- 🎯 **Context understanding** - AI reads between the lines
- 📊 **Actionable insights** - Route issues to the right teams

Perfect for customer support, content moderation, and data triage workflows.

In [ ]:
import fenic as fc

# ⚡ Configure for smart filtering
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="smart_filtering_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "filter": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000),
            # "filter": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "filter": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Smart filtering session configured")

## 📝 Step 1: Mixed Customer Feedback

Real customer messages that are hard to categorize with traditional filters:

In [ ]:
# 📝 Customer feedback with mixed sentiments and intents
feedback = session.create_dataframe([
    {"id": "FB001", "customer": "Sarah M.", "message": "This app crashes every time I try to export data. I've lost hours of work! This is completely unacceptable.", "rating": 1},
    {"id": "FB002", "customer": "Mike R.", "message": "Love the new dashboard! Would be amazing if you could add a dark mode option though.", "rating": 4},
    {"id": "FB003", "customer": "Jessica L.", "message": "The customer support team was incredibly helpful. They resolved my billing issue in minutes.", "rating": 5},
    {"id": "FB004", "customer": "Tom H.", "message": "Why can't I filter by multiple categories? Every other tool has this basic feature. Very frustrating.", "rating": 2},
    {"id": "FB005", "customer": "Lisa K.", "message": "Great product overall! Could you add integration with Slack? That would make my workflow perfect.", "rating": 4},
    {"id": "FB006", "customer": "David P.", "message": "Been using this for 3 years. Solid, reliable, does exactly what I need. No complaints here.", "rating": 5},
    {"id": "FB007", "customer": "Emma S.", "message": "The recent update broke my saved reports. Now I have to recreate everything from scratch. Terrible.", "rating": 1},
    {"id": "FB008", "customer": "Alex C.", "message": "Interface is clean and intuitive. Maybe add some advanced analytics features for power users?", "rating": 4}
])

print("📝 Customer Feedback - Mixed sentiments and requests:")
feedback.show()

## 🔍 Step 2: Smart Filter - Find Frustrated Customers

Use AI to identify customers who sound frustrated, regardless of their exact words:

In [ ]:
# 🔍 Smart filter: Find frustrated customers (AI understands tone and context)
frustrated_customers = feedback.filter(
    fc.semantic.predicate(
        "Customer message: {{ message }}\nThis customer sounds frustrated, angry, or upset about a problem or issue with the product",
        message=fc.col("message"),
        model_alias="filter"
    )
).cache()

print("😤 FRUSTRATED CUSTOMERS IDENTIFIED:")
frustrated_result = frustrated_customers.select("customer", "message", "rating")
frustrated_result.show()

frustrated_count = frustrated_customers.count()
print(f"\n📊 Found {frustrated_count} frustrated customers out of {feedback.count()} total feedback")

## 💡 Step 3: Smart Filter - Feature Requests

Identify feature requests and suggestions, even when phrased as compliments:

In [ ]:
# 💡 Smart filter: Find feature requests (AI detects suggestions and wishes)
feature_requests = feedback.filter(
    fc.semantic.predicate(
        "Customer message: {{ message }}\nThis message contains a feature request, suggestion, or wish for new functionality",
        message=fc.col("message"),
        model_alias="filter"
    )
).cache()

print("💡 FEATURE REQUESTS DETECTED:")
feature_result = feature_requests.select("customer", "message", "rating")
feature_result.show()

feature_count = feature_requests.count()
print(f"\n📊 Found {feature_count} feature requests hidden in feedback")

## 🌟 Step 4: Smart Filter - Praise and Loyalty

Find customers expressing genuine satisfaction and loyalty:

In [ ]:
# 🌟 Smart filter: Find loyal, satisfied customers
loyal_customers = feedback.filter(
    fc.semantic.predicate(
        "Customer message: {{ message }}\nThis customer expresses genuine satisfaction, loyalty, or praise for the product or service",
        message=fc.col("message"),
        model_alias="filter"
    )
).cache()

print("🌟 LOYAL CUSTOMERS IDENTIFIED:")
loyal_result = loyal_customers.select("customer", "message", "rating")
loyal_result.show()

loyal_count = loyal_customers.count()
print(f"\n📊 Found {loyal_count} genuinely satisfied customers")

print("\n🎯 SMART FILTERING RESULTS:")
print(f"   • Frustrated customers: {frustrated_count} (need immediate attention)")
print(f"   • Feature requests: {feature_count} (product development insights)")
print(f"   • Loyal customers: {loyal_count} (potential testimonials/case studies)")

print("\n💡 BUSINESS BENEFITS:")
print("   • Automatic issue triage based on emotional tone")
print("   • Hidden feature requests discovered in positive feedback")
print("   • Loyal customers identified for retention programs")
print("   • Context-aware filtering beyond keyword matching")

In [ ]:
session.stop()